In [15]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [17]:
# parameters
learning_rate = 1e-3
training_epochs = 15
batch_size = 100
drop_prob = 0.3      # dropout 을 적용하는 확률

In [18]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIS_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIS_data/',
                          train=False,
                          transform=transforms.ToTensor(),
                          download=True)

In [19]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [20]:
# nn layers
# input layer(1), hidden layer(4), output layer(1)
linear1 = torch.nn.Linear(784, 512, bias=True)
linear2 = torch.nn.Linear(512, 512, bias=True)
linear3 = torch.nn.Linear(512, 512, bias=True)
linear4 = torch.nn.Linear(512, 512, bias=True)
linear5 = torch.nn.Linear(512, 10, bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)         # drop_pob 의 확률로 dropout 을 적용

In [21]:
# xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0565,  0.0423, -0.0155,  ...,  0.1012,  0.0459, -0.0191],
        [ 0.0772,  0.0452, -0.0638,  ...,  0.0476, -0.0638,  0.0528],
        [ 0.0311, -0.1023, -0.0701,  ...,  0.0412, -0.1004,  0.0738],
        ...,
        [ 0.0334,  0.0187, -0.1021,  ...,  0.0280, -0.0583, -0.1018],
        [-0.0506, -0.0939, -0.0467,  ..., -0.0554, -0.0325,  0.0640],
        [-0.0183, -0.0123,  0.1025,  ..., -0.0214,  0.0220, -0.0741]],
       requires_grad=True)

In [22]:
# model 설정
model = torch.nn.Sequential(linear1, relu, dropout,
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4, relu, dropout,
                            linear5).to(device)

In [23]:
# define cost / loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [24]:
total_batch = len(data_loader)
model.train()     # dropout=True
for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.view(-1, 28 * 28).to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('Epoch {:6d}/{} Cost: {:.4f}'.format(
      epoch, training_epochs, avg_cost
  ))
print('Learning finished')

Epoch      0/15 Cost: 0.3106
Epoch      1/15 Cost: 0.1419
Epoch      2/15 Cost: 0.1119
Epoch      3/15 Cost: 0.0931
Epoch      4/15 Cost: 0.0817
Epoch      5/15 Cost: 0.0744
Epoch      6/15 Cost: 0.0680
Epoch      7/15 Cost: 0.0636
Epoch      8/15 Cost: 0.0570
Epoch      9/15 Cost: 0.0535
Epoch     10/15 Cost: 0.0516
Epoch     11/15 Cost: 0.0512
Epoch     12/15 Cost: 0.0501
Epoch     13/15 Cost: 0.0457
Epoch     14/15 Cost: 0.0427
Learning finished


In [28]:
# Test the model and check accuracy
with torch.no_grad():
  model.eval()      # dropout=False

  X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy: ',accuracy.item())

  r = random.randint(0, len(mnist_test)-1)
  X_single_data = mnist_test.test_data[r:r+1].view(-1, 28 * 28).float().to(device)
  Y_single_data = mnist_test.test_labels[r:r+1].to(device)

  print('Label: ',Y_single_data.item())
  single_prediction = model(X_single_data)
  print('Prediction: ',torch.argmax(single_prediction,1).item())

Accuracy:  0.9803999662399292
Label:  8
Prediction:  8


/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
